<a href="https://colab.research.google.com/github/alexyev/pennylane_projects/blob/master/Simulating_Noisy_Circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np

In [8]:
dev = qml.device('default.mixed', wires=2)

@qml.qnode(dev)
def circuit():
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0, 1])
  return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

print(f'QNode output: {circuit():.4f}')

QNode output: 1.0000


In [9]:
print(f'Output state is: \n{np.real(dev.state)}')

Output state is: 
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]


In [10]:
# simualting noise in a bitflip circuit

@qml.qnode(dev)
def bitflip_circuit(p):
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0, 1])
  qml.BitFlip(p, wires=0)
  qml.BitFlip(p, wires=1)
  return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

ps = [0.001, 0.01, 0.1, 0.2]

for p in ps:
  print(f'Qnode output for BitFlip probability {p} is: {bitflip_circuit(p):.4f}')

Qnode output for BitFlip probability 0.001 is: 0.9960
Qnode output for BitFlip probability 0.01 is: 0.9604
Qnode output for BitFlip probability 0.1 is: 0.6400
Qnode output for BitFlip probability 0.2 is: 0.3600


In [11]:
print(f'Output state for bitflip probability {p} is: \n{np.real(dev.state)}')

Output state for bitflip probability 0.2 is: 
[[0.34 0.   0.   0.34]
 [0.   0.16 0.16 0.  ]
 [0.   0.16 0.16 0.  ]
 [0.34 0.   0.   0.34]]


In [12]:
# simualting depolarization in a Bell State

@qml.qnode(dev)
def depolarizing_circuit(p):
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0, 1])
  qml.DepolarizingChannel(p, wires=0)
  qml.DepolarizingChannel(p, wires=1)
  return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

ps = [0.001, 0.01, 0.1, 0.2]

for p in ps:
  print(f'Qnode output for depolarizing probability {p} is: {depolarizing_circuit(p):.4f}')

Qnode output for depolarizing probability 0.001 is: 0.9973
Qnode output for depolarizing probability 0.01 is: 0.9735
Qnode output for depolarizing probability 0.1 is: 0.7511
Qnode output for depolarizing probability 0.2 is: 0.5378


In [13]:
ev = np.tensor(0.7781, requires_grad=False)

def sigmoid(x):
  return 1/(1+np.exp(-x))

@qml.qnode(dev)
def damping_circuit(x):
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0, 1])
  qml.AmplitudeDamping(sigmoid(x), wires=0)
  qml.AmplitudeDamping(sigmoid(x), wires=1)
  return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [14]:
def cost(x, target):
  return (damping_circuit(x) - target) ** 2

In [16]:
opt = qml.GradientDescentOptimizer(stepsize = 10)
steps = 35
x = np.tensor(0.0, requires_grad=True)

for i in range(steps):
  (x, ev), cost_val = opt.step_and_cost(cost, x, ev)
  if i % 5 == 0 or i == steps - 1:
    print(f'Step: {i}    Cost Value: {cost_val}')

print(f'Qnode output after optimization: {damping_circuit(x):.4f}')
print(f'Experimental Expectation Output: {ev}')
print(f'Optimal noise parameter p: {sigmoid(x.take(0)):.4f}')

Step: 0    Cost Value: 0.07733961000000007
Step: 5    Cost Value: 0.07733960999999988
Step: 10    Cost Value: 0.07733960999863909
Step: 15    Cost Value: 0.07733960170319246
Step: 20    Cost Value: 0.07728907281668594
Step: 25    Cost Value: 0.006192562764640602
Step: 30    Cost Value: 6.427645677603198e-07
Step: 34    Cost Value: 1.1072988376257744e-09
Qnode output after optimization: 0.7781
Experimental Expectation Output: 0.7781
Optimal noise parameter p: 0.1271
